# Law QnA

In [8]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from tqdm import tqdm
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
import pandas as pd
import pinecone
import tiktoken
from uuid import uuid4

In [2]:
load_dotenv('.env')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [3]:
pinecone.init(      
	api_key=PINECONE_API_KEY,      
	environment='gcp-starter'      
)
index = pinecone.Index('law-search')

In [4]:
tokenizer = tiktoken.get_encoding('cl100k_base')
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [5]:
embed = OpenAIEmbeddings()

In [6]:
def load_chunk_persist_pdf(constitution_path='india_code/data/constitution.csv'):
    batch_limit = 64
    texts = []
    metadatas = []

    df = pd.read_csv(constitution_path)
    for i in tqdm(range(df.shape[0])):
        loader = PyPDFLoader(df.iloc[i,1])
        contents = "".join(page.page_content for page in loader.load())
        record_texts = text_splitter.split_text(contents)
        
        record_metadatas = [{
            "chunk": j, "text": text, "source": f'{df.iloc[i,0]}, part {j}'
        } for j, text in enumerate(record_texts)]
        texts.extend(record_texts)
        metadatas.extend(record_metadatas)
        if len(texts) >= batch_limit:
            ids = [str(uuid4()) for _ in range(len(texts))]
            embeds = embed.embed_documents(texts)
            for k in range(0, len(ids), batch_limit):
                stop_index = min(k+batch_limit, len(ids))
                index.upsert(vectors=zip(ids[k:stop_index], embeds[k:stop_index], metadatas[k:stop_index]))
            texts = []
            metadatas = []
        if i>=0:
            break

    if texts:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))

load_chunk_persist_pdf()

  0%|          | 0/115 [01:46<?, ?it/s]


In [9]:
text_field = "text"

vectorstore = Pinecone(
    index,
    embed.embed_query,
    text_field
)

c:\Users\Pratik Sanghavi\Desktop\Projects\Web-Scraping\03. Web_Scraping_ScraPy\Scrapy_Venv\Lib\site-packages\langchain_community\vectorstores\pinecone.py:73: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [35]:
query = "Which are the relevant regulations related to Emergency?"
vectorstore.similarity_search(
    query, 
    k=15
)

[Document(page_content='217\n1[(1B) Nothing in clause (1A) shall apply— \n(a) to any law which does not contain a recital to the effect that \nsuch law is in relation to the Proclamation of Emergency in operation \nwhen it is made; or \n(b) to any executive action taken otherwise than under a law \ncontaining such a recital.]', metadata={'chunk': 1790.0, 'name': 'The Constitution of India'}),
 Document(page_content='such executive action may be taken, under this article in relation to or in any \nState or Union territory in which or in any part of which the Proclamation of \nEmergency is not in operation, if and in so far as the security of India or any \npart of the territory thereof is threatened by activities in or in relation to the part \nof the territory of India in which the Proclamation of Emergency is in \noperation.]', metadata={'chunk': 1777.0, 'name': 'The Constitution of India'}),
 Document(page_content='executive action may be taken, under this article in relation to or i

In [11]:
llm = ChatOpenAI()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k":15})
)

c:\Users\Pratik Sanghavi\Desktop\Projects\Web-Scraping\03. Web_Scraping_ScraPy\Scrapy_Venv\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [49]:
qa.run(query)

'Based on the provided context, the relevant regulations related to Emergency are not specified.'

In [48]:
qa.run("Are fundamental rights guaranteed during Emergency?")

'No, during a Proclamation of Emergency, the President may suspend the enforcement of certain fundamental rights as stated in Article 359 of the Constitution of India. However, some fundamental rights like the right to life and personal liberty (Article 21) and the right against exploitation (Article 23) cannot be suspended even during an Emergency.'

In [50]:
qa.run("What is stated with regards to Hindu temples?")

'According to the provided context, there is a mention of funds being charged and paid out of the Consolidated Fund of the State of Tamil Nadu for the maintenance of Hindu temples and shrines in the territories transferred to that state. However, there is no specific information regarding the management or regulation of Hindu temples.'

In [51]:
qa.run("what are the limitations on the right to freedom of expression?")

'The limitations on the right to freedom of expression are mentioned in Article 19 of the Constitution. The law imposes reasonable restrictions on the exercise of the right to freedom of speech and expression in the interests of the sovereignty and integrity of India, the security of the State, friendly relations with foreign States, public order, decency or morality, contempt of court, defamation, or incitement to an offense. Additionally, the freedom of speech and expression does not give the right to make statements that may incite violence or promote hatred.'

In [52]:
qa.run("How are minorities empowered by the constitution of India?")

'Minorities in India are empowered by the constitution in several ways:\n\n1. Right to Establish and Administer Educational Institutions: The constitution grants minorities the right to establish and administer educational institutions of their choice. This ensures that minority communities can preserve and promote their distinct language, script, and culture.\n\n2. Equality of Opportunity: The constitution guarantees equality of opportunity for all citizens in matters relating to employment or appointment to any office under the state. No citizen can be discriminated against on the grounds of religion, race, caste, sex, descent, place of birth, or residence.\n\n3. Prohibition of Discrimination: The constitution prohibits discrimination on the grounds of religion, race, caste, sex, and place of birth. This ensures that minorities are protected from any form of discrimination and are treated equally before the law.\n\n4. Special Officer for Linguistic Minorities: The constitution mandat

In [53]:
qa.run("Does the constitution have any appeasement policies for minorities? Anything that disadvantages certain groups?")

'Yes, the Constitution of India does have provisions for the reservation of seats and special representation for certain groups, such as Scheduled Castes, Scheduled Tribes, and socially and educationally backward classes of citizens. These provisions aim to provide equal opportunities and representation for these groups in various fields, including education and employment. However, it is important to note that these provisions are meant to address historical disadvantages faced by these groups and promote social justice, rather than disadvantage any particular group.'

In [54]:
qa.run("What are certain policies made with respect to Muslims and Christians?")

'Based on the given context, there are several policies mentioned in the Constitution of India that protect the rights of minorities, including Muslims and Christians. These policies include:\n\n1. Prohibition of discrimination: Article 15 of the Constitution prohibits discrimination on the grounds of religion, race, caste, sex, or place of birth. This means that no citizen can be discriminated against based solely on their religion, including Muslims and Christians.\n\n2. Protection of interests of minorities: Article 29 ensures the protection of the interests of minorities. It grants minorities the right to conserve their distinct language, script, and culture. It also prohibits the denial of admission to any educational institution based on religion, race, caste, language, or any other grounds.\n\n3. Freedom of religion: Article 25 guarantees freedom of conscience and the right to freely profess, practice, and propagate religion. This means that individuals, including Muslims and Ch

In [12]:
qa_with_source = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k":15})
)

In [13]:
query = "Which are the relevant regulations related to Emergency?"
qa_with_source(query)

c:\Users\Pratik Sanghavi\Desktop\Projects\Web-Scraping\03. Web_Scraping_ScraPy\Scrapy_Venv\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'question': 'Which are the relevant regulations related to Emergency?',
 'answer': 'The relevant regulations related to Emergency are outlined in the Constitution of India. These include provisions such as Article 352, which deals with the proclamation of Emergency, and Article 359, which allows for the suspension of the enforcement of rights conferred by Part III during emergencies. Additionally, other articles such as Article 356, Article 358, and Article 360 also address various aspects of emergency provisions. \n',
 'sources': 'The Constitution of India, parts 1790, 1777, 1787, 1242, 112, 1783, 1781, 113, 1742, 1717, 1778, 1743, 1773, 2270, 2639'}

In [14]:
qa_with_source("What are certain policies made with respect to Muslims and Christians? Are there any special carve outs or benefits accorded to them?")

{'question': 'What are certain policies made with respect to Muslims and Christians? Are there any special carve outs or benefits accorded to them?',
 'answer': 'The Constitution of India prohibits discrimination on the grounds of religion, race, caste, sex, or place of birth. It ensures equality of opportunity for all citizens in matters relating to employment or appointment to any office under the State. There are no special carve outs or benefits accorded to Muslims or Christians specifically. However, the Constitution does allow for special provisions to be made for the advancement of socially and educationally backward classes of citizens, as well as for Scheduled Castes and Scheduled Tribes. (The Constitution of India, part 181, part 13, part 170, part 10, part 238, part 8, part 12, part 106, part 102, part 104, part 114, part 237, part 233, part 1996, part 173)\n',
 'sources': 'The Constitution of India'}

In [15]:
qa_with_source("Do you think there's any section that disadvantages Hindus?")

{'question': "Do you think there's any section that disadvantages Hindus?",
 'answer': 'There is no section that specifically disadvantages Hindus in the provided excerpts.\n',
 'sources': 'The Constitution of India, part 169, part 232, part 170, part 191, part 173, part 181, part 1075, part 238, part 233, part 1716, part 297, part 231, part 228, part 237, part 1152'}